In [ ]:
from cloudvolume import CloudVolume
from cloudvolume.exceptions import SkeletonDecodeError
from itertools import islice
from skimage.transform import downscale_local_mean
import napari
from skimage import io
import random
import h5py
from skimage import measure
from brainlit.preprocessing import removeSmallCCs
import numpy as np 
import matplotlib.pyplot as plt 
import subprocess
import tables
from napari_animation import AnimationWidget
from tqdm import tqdm
import pickle
import networkx as nx
import seaborn as sns
import pandas as pd
import brainrender
from brainlit.utils.session import NeuroglancerSession
from skimage.filters import threshold_otsu, threshold_local
from brainlit.preprocessing import split_frags, rename_states_consecutively, label_points
from brainlit.algorithms.connect_fragments import most_probable_neuron_path
from brainlit.algorithms.connect_fragments import trace_evaluation
from brainlit.algorithms.connect_fragments import dynamic_programming_viterbi2
import similaritymeasures
from brainlit.viz import Bresenham3D
from cloudvolume import Skeleton
from sklearn.metrics import pairwise_distances_argmin_min
import time
import zarr
from brainlit.preprocessing import image_process
%gui qt5

In [ ]:
dir = "s3://open-neurodata/brainlit/brain1"
dir_segments = "s3://open-neurodata/brainlit/brain1_segments"

ngl_sess = NeuroglancerSession(mip = 0, url = dir, url_segments=dir_segments)

res = [0.3,0.3,1]
threshold = 0.9

block_size = 101

img, bbox, vox = ngl_sess.pull_vertex_list(2, [0], [1,1,1])
bbox

In [ ]:
dir = "s3://open-neurodata/brainlit/brain1"

vol = CloudVolume(dir, parallel=1, mip=0, fill_missing=True)
vol.shape

In [ ]:
#view
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(img)
viewer.add_points(vox, size=7)
napari.run()

In [ ]:
io.imsave("/Users/thomasathey/Documents/mimlab/mouselight/input/images/gui/500.tif", img)

In [ ]:
pred_path ="/Users/thomasathey/Documents/mimlab/mouselight/input/images/gui/250_Probabilities.h5"
f = h5py.File(pred_path, 'r')
pred = f.get('exported_data')
pred = pred[:,:,:,1]
im_processed = pred

mask = im_processed >= threshold
labels = measure.label(mask)

soma_coords = vox[0:1,:]

In [ ]:
new_labels = split_frags(soma_coords, labels, im_processed, threshold, res)
_, soma_lbls = label_points(new_labels, soma_coords, res)
soma_lbl = soma_lbls[0]

In [ ]:
viterbi = most_probable_neuron_path(image=img.astype(float), labels=new_labels, soma_lbls=[soma_lbl], resolution=res, coef_dist=10, coef_curv=1000)
viterbi.frags_to_lines()
viterbi.reset_dists(type="all")
viterbi.compute_all_costs_dist(point_point_func=viterbi.point_point_dist, point_blob_func=viterbi.point_blob_dist)
viterbi.compute_all_costs_int()
viterbi.create_nx_graph()

In [ ]:
with open('/Users/thomasathey/Documents/mimlab/mouselight/input/images/gui/viterbi_250.pickle', 'wb') as handle:
    pickle.dump(viterbi, handle)

In [ ]:
from brainlit.utils.Neuron_trace import NeuronTrace

nt = NeuronTrace(path="/Users/thomasathey/Documents/mimlab/mouselight/input/mim_slides/SNT_Data.swc")
df = nt.get_df()
df[['x','y','z']].to_numpy()

In [ ]:
im = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/input/images/gui/500.tif")
im2 = im.astype("float64")

In [ ]:
z = zarr.zeros(im.shape, chunks=(100,100,100), dtype='uint16')

In [ ]:
z[:,:,:] = im

In [ ]:
zarr.save("/Users/thomasathey/Documents/mimlab/mouselight/input/images/big/500.zarr", z)

In [ ]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(z, name="image", scale=[0.3,0.3,1])

In [ ]:
from skimage.transform import pyramid_gaussian
multiscale = list(pyramid_gaussian(im2, downscale=2, max_layer=4))
print('multiscale level shapes: ', [p.shape for p in multiscale])
print(f"Original dtype: {im2.dtype}")
print(f"New dtype: {multiscale[0].dtype}")

In [ ]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(multiscale, rgb=False, multiscale=True, name="multiscale image", scale=[0.3,0.3,1])
viewer.add_image(multiscale[0], rgb=False, name="image", scale=[0.3,0.3,1])

## Make viterbi object

In [ ]:
im_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/images/2018-08-01_0_first10_quantitative.tif"
probs_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/2018-08-01_0_first10_quantitative_Probabilities.h5"
res = [0.3,0.3,1]

im_og = io.imread(im_path, plugin="tifffile")

f = h5py.File(probs_path, 'r')
pred = f.get('exported_data')
pred = pred[:,:,:,1]
im_processed = pred

threshold = 0.9  # 0.1
labels = measure.label(im_processed > threshold)
im_og.shape

In [ ]:
im_og.dtype

In [ ]:
viewer = napari.Viewer(ndisplay=3)

viewer.add_image(im_og)
viewer.add_labels(labels)
viewer.add_points(soma_coords, face_color='red')



In [ ]:
z = zarr.zeros(im_og.shape, chunks=im_og.shape, dtype='uint16')
z[:,:,:] = im_og
im_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/octopus/image_0.zarr"
zarr.save(im_path, z)

z = zarr.zeros(pred.shape, chunks=pred.shape, dtype='float')
z[:,:,:] = pred
prob_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/octopus/probs_0.zarr"
zarr.save(prob_path, z)

soma_coords = [[366,237, 122], [234,462, 122], [227, 641, 92]]

In [ ]:
from brainlit.algorithms.generate_fragments import state_generation
sg = state_generation(
    im_path,
    "/Applications/ilastik-1.3.3post3-OSX.app/Contents/ilastik-release/run_ilastik.sh",
    "/Users/thomasathey/Documents/mimlab/mouselight/octopus_experiment/octopus_exp.ilp",
    chunk_size=im_og.shape,
    soma_coords=soma_coords,
    parallel=1,
    prob_path = prob_path)
sg.compute_frags()
sg.compute_image_tiered()
sg.compute_soma_lbls()
sg.compute_states()
sg.compute_edge_weights()

In [ ]:

labels_path = im_path.split(".")[0] + "_labels.zarr"
z = zarr.open(labels_path)
new_labels = z[:,:,:]



In [ ]:
viewer = napari.Viewer(ndisplay=3)

viewer.add_image(im_og)
viewer.add_labels(labels)
viewer.add_labels(new_labels)

In [ ]:
with open("/Users/thomasathey/Documents/mimlab/mouselight/input/images/octopus/viterbi_0_multisoma.pickle", "rb") as handle:
    v = pickle.load(handle)

In [ ]:
from brainlit.algorithms.connect_fragments.viterbrain import ViterBrain
# with open(sg.states_path, "rb") as handle:
#     G = pickle.load(handle)
viterbrain = ViterBrain(
            v.nxGraph,
            v.tiered_path,
            fragment_path=v.fragment_path,
            resolution=v.resolution,
            coef_curv=1000,
            coef_dist=10,
            coef_int=1,
            parallel=6,
        )
viterbrain.compute_all_costs_dist(
            frag_frag_func=viterbrain.frag_frag_dist,
            frag_soma_func=viterbrain.frag_soma_dist,
        )
viterbrain.compute_all_costs_int()

In [ ]:
with open('/Users/thomasathey/Documents/mimlab/mouselight/input/images/octopus/viterbi_0_multisoma.pickle', 'wb') as handle:
    pickle.dump(viterbrain, handle)

## Find fragment coords

In [16]:
import pickle
import zarr
import numpy as np 
import napari
path = (
    "/Users/thomasathey/Documents/mimlab/mouselight/input/images/octopus/viterbi_"
    + str(0)
    + "_multisoma.pickle"
)

with open(path, "rb") as handle:
    viterbi = pickle.load(handle)

z = zarr.open(viterbi.fragment_path, mode="r")
frags = z[:,:,:]

In [28]:
mask = frags == 12

viewer = napari.Viewer(ndisplay=3)
viewer.add_image(mask, name="image")

<Image layer 'image' at 0x16a2edfd0>

In [32]:
coords = []
for frag in [2, 3, 4, 5, 7, 8, 9, 10, 11, 12]:
    coords.append(np.argwhere(frags == frag))
print(coords)

with open("/Users/thomasathey/Desktop/frags.pkl", "wb") as handle:
    viterbi = pickle.dump(coords, handle)

[array([[  0, 281, 123],
       [  0, 281, 124],
       [  0, 282, 124],
       [  0, 283, 123],
       [  0, 283, 126],
       [  1, 281, 123],
       [  1, 281, 124],
       [  1, 282, 123],
       [  1, 282, 124],
       [  1, 282, 125],
       [  1, 283, 123],
       [  1, 283, 124],
       [  1, 283, 125],
       [  2, 282, 123],
       [  2, 282, 125],
       [  3, 281, 124],
       [  3, 281, 125],
       [  3, 282, 123],
       [  3, 282, 124],
       [  3, 282, 126],
       [  4, 279, 124],
       [  4, 279, 125],
       [  4, 280, 124],
       [  4, 281, 124],
       [  4, 281, 125],
       [  4, 282, 122],
       [  4, 282, 123],
       [  4, 282, 124],
       [  5, 278, 124],
       [  5, 278, 125],
       [  5, 279, 124],
       [  5, 279, 125],
       [  5, 280, 124],
       [  5, 280, 125],
       [  5, 281, 122],
       [  5, 281, 123],
       [  5, 281, 124],
       [  5, 282, 122],
       [  5, 282, 123],
       [  5, 282, 124],
       [  6, 278, 123],
       [  6, 27

In [33]:
with open("/Users/thomasathey/Desktop/frags.pkl", "rb") as handle:
    coords = pickle.load(handle)

print(coords[0])

[[  0 281 123]
 [  0 281 124]
 [  0 282 124]
 [  0 283 123]
 [  0 283 126]
 [  1 281 123]
 [  1 281 124]
 [  1 282 123]
 [  1 282 124]
 [  1 282 125]
 [  1 283 123]
 [  1 283 124]
 [  1 283 125]
 [  2 282 123]
 [  2 282 125]
 [  3 281 124]
 [  3 281 125]
 [  3 282 123]
 [  3 282 124]
 [  3 282 126]
 [  4 279 124]
 [  4 279 125]
 [  4 280 124]
 [  4 281 124]
 [  4 281 125]
 [  4 282 122]
 [  4 282 123]
 [  4 282 124]
 [  5 278 124]
 [  5 278 125]
 [  5 279 124]
 [  5 279 125]
 [  5 280 124]
 [  5 280 125]
 [  5 281 122]
 [  5 281 123]
 [  5 281 124]
 [  5 282 122]
 [  5 282 123]
 [  5 282 124]
 [  6 278 123]
 [  6 278 124]
 [  6 278 125]
 [  6 278 126]
 [  6 279 123]
 [  6 280 123]
 [  6 280 124]
 [  6 280 125]
 [  6 281 122]
 [  6 281 123]
 [  6 281 124]
 [  7 277 123]
 [  7 277 124]
 [  7 277 125]
 [  7 277 126]
 [  7 278 123]
 [  7 278 124]
 [  7 278 125]
 [  7 278 126]
 [  7 278 127]
 [  7 279 122]
 [  7 279 123]
 [  7 279 126]
 [  7 280 123]
 [  7 280 124]
 [  7 280 125]
 [  8 277 